# Bayes ingenuo

Este programa clasifica correos electrónicos como spam o ham utilizando el alrogítmo de bayes ingenuo.

setwd("C:/Users/Yosshua/Desktop/ITAM/VERANO_2021/ML/machine-learning-2021-2/supervisado/bayes_ingenuo")

In [1]:
usePackage <- function(p) 
{
  if (!is.element(p, installed.packages()[,1]))
    install.packages(p, repos = "https://cran.itam.mx/")
  suppressPackageStartupMessages(require(p, character.only = TRUE, quietly  = TRUE))
}

In [2]:
usePackage('R.utils')
usePackage('tm')
usePackage("tidyr")

## Descarga los datos 

In [3]:
download.mails <- function(url, dir_name, file_name){

  if (!file.exists(dir_name)) {
    dir.create(dir_name)  
  }
  
  download.file(url, destfile=file.path(dir_name, paste0(file_name,".tar.bz2")) )
  bunzip2(file.path(dir_name, paste0(file_name,".tar.bz2")))
  
  untar(file.path(dir_name, paste0(file_name,".tar")), exdir = dir_name)
  
  if (file.exists(file.path(dir_name, paste0(file_name,".tar")))) {
    file.remove(file.path(dir_name, paste0(file_name,".tar")))
  }
  
}

In [4]:
dir_name <- "data"
file_name <- "easy_ham_2"
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2"

download.mails(url, dir_name, file_name)


[1] TRUE

In [5]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2"
file_name <- "hard_ham"

download.mails(url, dir_name, file_name)

[1] TRUE

In [6]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_spam_2.tar.bz2"
file_name <- "spam_2"

download.mails(url, dir_name, file_name)

[1] TRUE

## Preprocesamiento de los correos electrónicos

In [7]:
# Hacemos una función que leea el mensaje del archivo que se le pase como parámetro
# asumimos que el archivo contiene un correo

lee_mensaje <- function(correo) {
  fd <- file(correo, open = "rt")
  lineas <- readLines(fd, warn=FALSE)
  close(fd)
  mensaje <- lineas[seq(which(lineas == "")[1] + 1, length(lineas), 1)]
  return (paste(mensaje, collapse = "\n"))
}

In [8]:
# Creamos variables con los directorios donde se encuentran los datos
dir_name <- "data"
trayectoria_spam     <- file.path(dir_name, "spam_2")
trayectoria_easyham  <- file.path(dir_name, "easy_ham_2")
trayectoria_hardham  <- file.path(dir_name, "hard_ham")
# spam_2: 1397 spam messages.
# easy_ham_2: 1400 non-spam messages. 
# hard_ham: 250 non-spam messages


### Spam

In [9]:
# Leemos el directorio donde se encuentran los correos clasificados como spam
archivos_correos_spam <- dir(trayectoria_spam)

# quitamos el guión llamado cmds
archivos_correos_spam <- archivos_correos_spam[which(archivos_correos_spam!="cmds")] #[1:250]

todo_spam <- sapply(archivos_correos_spam,
                   function(p) lee_mensaje(file.path(trayectoria_spam, p)))
                    
todo_spam <- enc2utf8(todo_spam)

### Easy ham

In [10]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_easy_ham <- dir(trayectoria_easyham)

# quitamos el guión llamado cmds
archivos_correos_easy_ham <- archivos_correos_easy_ham[which(archivos_correos_easy_ham!="cmds")] #[1:250]

todo_easy_ham <- sapply(archivos_correos_easy_ham,
                    function(p) lee_mensaje(file.path(trayectoria_easyham, p)))

todo_easy_ham <- enc2utf8(todo_easy_ham)

## Preparación de corpus y bolsa de palabras

In [11]:
obtiene_TermDocumentMatrix <- function (vector_correos) {
  control <- list(stopwords = TRUE,
                removePunctuation = TRUE,
                removeNumbers = TRUE,
                minDocFreq = 2)
  corpus <- Corpus(VectorSource(vector_correos))
  return(TermDocumentMatrix(corpus, control))
}

### Spam

In [12]:
spam_TDM <- obtiene_TermDocumentMatrix(todo_spam)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento SPAM
matriz_spam <- as.matrix(spam_TDM)

conteos_spam <- rowSums(matriz_spam)
df_spam <- data.frame(cbind(names(conteos_spam),
                            as.numeric(conteos_spam)),
                      stringsAsFactors = FALSE)
names(df_spam) <- c("terminos", "frecuencia")
df_spam$frecuencia <- as.numeric(df_spam$frecuencia)
ocurrencias_spam <- sapply(1:nrow(matriz_spam),
                          function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                          {
                            length(which(matriz_spam[i, ] > 0)) / ncol(matriz_spam)
                          })
densidad_spam <- df_spam$frecuencia/sum(df_spam$frecuencia,na.rm = TRUE)

df_spam <- transform(df_spam,
                     densidad = densidad_spam,
                     ocurrencias = ocurrencias_spam)

In [13]:
head(df_spam)

,terminos,frecuencia,densidad,ocurrencias
,<chr>,<dbl>,<dbl>,<dbl>
1,abandoned,12,3.076907e-05,0.006442377
2,accept,89,2.282040e-04,0.050823193
3,address,815,2.089733e-03,0.243378669
4,agree,14,3.589725e-05,0.006442377
5,agreed,21,5.384588e-05,0.012884753
6,also,487,1.248712e-03,0.156048676


### Easy ham

In [14]:
easy_ham_TDM <- obtiene_TermDocumentMatrix(todo_easy_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham
matriz_easy_ham <- as.matrix(easy_ham_TDM)

conteos_easy_ham <- rowSums(matriz_easy_ham)
df_easy_ham <- data.frame(cbind(names(conteos_easy_ham),
                            as.numeric(conteos_easy_ham)),
                      stringsAsFactors = FALSE)
names(df_easy_ham) <- c("terminos", "frecuencia")
df_easy_ham$frecuencia <- as.numeric(df_easy_ham$frecuencia)
ocurrencias_easy_ham <- sapply(1:nrow(matriz_easy_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_easy_ham[i, ] > 0)) / ncol(matriz_easy_ham)
                           })
densidad_easy_ham <- df_easy_ham$frecuencia/sum(df_easy_ham$frecuencia,na.rm = TRUE)

df_easy_ham <- transform(df_easy_ham,
                     densidad = densidad_easy_ham,
                     ocurrencias = ocurrencias_easy_ham)

In [15]:
head(df_easy_ham)

,terminos,frecuencia,densidad,ocurrencias
,<chr>,<dbl>,<dbl>,<dbl>
1,actually,195,9.992826e-04,0.11285714
2,adding,35,1.793584e-04,0.02071429
3,addition,32,1.639848e-04,0.02000000
4,additional,40,2.049810e-04,0.02285714
5,ages,16,8.199242e-05,0.01142857
6,ago,109,5.585733e-04,0.06571429


## Cálculo de probabilidad a posteriori

In [16]:
a_posteriori <- function(trayectoria, df_entrenamiento, a_priori = 0.5, c = 1e-6)
{
  mensaje <- lee_mensaje(trayectoria)
  mensaje <- enc2utf8(mensaje)
  mensaje_TDM <- obtiene_TermDocumentMatrix(mensaje)
  conteos_mensaje <- rowSums(as.matrix(mensaje_TDM))
    
  # Encuentra palabras en data frame de entrenamiento
  mensaje_palabras_comunes <- intersect(names(conteos_mensaje), df_entrenamiento$terminos)
  index <- order(df_entrenamiento$densidad)
  nuevoDF <- df_entrenamiento[index,]  
  
  # Ahora sólo aplicamos la clasificación Bayes ingenuo
  if(length(mensaje_palabras_comunes) < 1)
  {
    #return(a_priori * c ^ (length(conteos_mensaje)))
    return( list( log(a_priori) + (length(conteos_mensaje)) *log(c), c("")))
  }
  else
  {    

    probabilidades_palabras_comunes <- nuevoDF$densidad[match(mensaje_palabras_comunes, nuevoDF$terminos)]
          
    # Obtenemos las palabras con mayor probabilidad de aparecer que están en el conjunto de entrenamiento y en el correo.
    # Y seleccionamos las 3 más significativas.
    ans <-   nuevoDF$terminos[match(mensaje_palabras_comunes, nuevoDF$terminos)]
    ans <-  ans[ max(1, length(ans)-2):length(ans) ]    
      
    #return(a_priori * prod(probabilidades_palabras_comunes) * c ^ (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
    return (list(log(a_priori) + sum(log(probabilidades_palabras_comunes)) + log(c) * (length(conteos_mensaje) - length(mensaje_palabras_comunes)), 
                 ans))
    
  }
}

## Clasificación

In [17]:
#Regresa verdadero si es spam y falso en caso contrario.
clasifica_spam <- function(trayectoria, archivos) {

  hard_ham_spam_prueba <- sapply(archivos,
                             function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
  hard_ham_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_easy_ham))
                
                                
  return (list(ifelse(as.numeric(hard_ham_spam_prueba[1,]) > as.numeric(hard_ham_ham_prueba[1,]),
                 TRUE, FALSE), hard_ham_spam_prueba[2,], hard_ham_ham_prueba[2,]))
    
}

### Hard ham

In [18]:
# Leemos el directorio donde se encuentran los correos clasificados como ham dificilmente identificables
archivos_correos_hard_ham <- dir(trayectoria_hardham)

# quitamos el guión llamado cmds
archivos_correos_hard_ham <- archivos_correos_hard_ham[which(archivos_correos_hard_ham!="cmds")]

In [19]:
# Clasificamos los diferentes conjuntos
hard_ham_res <- clasifica_spam(trayectoria_hardham, archivos_correos_hard_ham)
easy_ham_res <- clasifica_spam(trayectoria_easyham, archivos_correos_easy_ham)
spam_res     <- clasifica_spam(trayectoria_spam,    archivos_correos_spam)

## Resultados

In [20]:
# Nombres de correos que fueron mal clasificados

# Todos los archivos de hard ham son correos normales
hard_ham_res_1 <- unlist(hard_ham_res[1], use.names = FALSE)
print("Nombres de hard ham mal clasificados")
archivos_correos_hard_ham[hard_ham_res_1]

# Todos los archivos de easy_ham son correos normales
easy_ham_res_1 <- unlist(easy_ham_res[1], use.names = FALSE)
print("Nombres de easy ham mal clasificados")
archivos_correos_easy_ham[easy_ham_res_1]

# Todos los archivos de esta ruta son spam
spam_res_1 <- unlist(spam_res[1], use.names = FALSE)
print("Nombres de spam mal clasificados")
archivos_correos_spam[!spam_res_1]


[1] "Nombres de hard ham mal clasificados"


[1] "00002.ca96f74042d05c1a1d29ca30467cfcd5"
  [2] "00003.268fd170a3fc73bee2739d8204856a53"
  [3] "00007.d24e99a602ee7fb442714c0d448cd08e"
  [4] "00008.b42457819236bee543bebffb61b91e44"
  [5] "00010.e82bd1f5f7eae426682a7f8e4cbf1ae6"
  [6] "00011.acdfa5be40e7b6c3ad3df28c63670c7c"
  [7] "00012.58a866f18474d94989984958e1789df4"
  [8] "00017.840244edb8cc88aba7129296ea536212"
  [9] "00018.75bf8472753f24aa22df72c7301e07ec"
 [10] "00020.eca3ca2c144cd6ceaf677f3d8b7b0eed"
 [11] "00021.1707ccb203e1a39f5167f1c0d65cc235"
 [12] "00022.66e4bce429ab25c5d2c7e8a1a38838a0"
 [13] "00024.9d2b87b00039e2527adfebd188dd3e98"
 [14] "00025.c27f4dd57c84c01305bd30bdee96810b"
 [15] "00026.bb0f14a06b51317f3fc7854cdaae5cdb"
 [16] "00027.87ab6708d16f330c0cb84c42a2adf154"
 [17] "00028.13ae4e6472d6f2ded79066b8a7472a27"
 [18] "00029.844ef4d2066aab3fdf93692142d5911f"
 [19] "00030.81e02cde7ddb4986b99b38a31ef8cf9a"
 [20] "00033.a09f363a94e0c632227ab453b9fd4908"
 [21] "00034.a3365d4376445f183171dad145d0b82d"
 [22] "00035.a0e0e8cdca0b8352a9e9c2c81e5d5cd7"
 [23] "00036.b2641f0e9ff64695cf368088640bee0c"
 [24] "00037.55830ddeb2e48be48787c1fb1656ee47"
 [25] "00038.a5f3ff0736ac35b3102c6712ffed9a60"
 [26] "00039.b2b936a8501444b213f61f9ff193b480"
 [27] "00040.78ac9f85875ccded71e3bbb55a7e6ac1"
 [28] "00041.aee0699a14f472a43d9a9d178aa21d70"
 [29] "00043.d8d12ae6c64ff65812f48641822d4892"
 [30] "00044.b2f03a4d512deb2c4702a3bd60a4fd88"
 [31] "00045.f1d1f852b14ac9cc7b8af57fed17e1dc"
 [32] "00046.026e41e68016ebba835a855d4e2af80f"
 [33] "00048.edce23084d9ebd5cd49924ba4bd2c344"
 [34] "00049.7ed9039cd4c9cb59c4be39fdeaca0c64"
 [35] "00050.22f61c80e6b78b95140773a3ef3848eb"
 [36] "00053.64ab5b8544333b7c16af8c87b5ebe800"
 [37] "00054.7e71da5e65cb85d5c483c3de59a5d63b"
 [38] "00055.d4984b38068ffd0b982eb1e5c117512f"
 [39] "00056.c17c95457d7524edd8a72b6de455c4d2"
 [40] "00059.ad3ed6777e73773ebd7ee5455e6a43e3"
 [41] "00060.bb5f0a4bb14bf8126f7024c538c22c92"
 [42] "00061.ced4949974bf2c9bb387310f53172c9b"
 [43] "00063.70680acb99a7bb45b2194c228388ed43"
 [44] "00064.a80b45bb0339ed948bbbd968c074c60e"
 [45] "00067.78f9a9d537907424c503f4d29a2c24b4"
 [46] "00070.a6fc1e36f88c0bd0e0749274082938fc"
 [47] "00071.c101f559f8fb74cad5caf665a3713c34"
 [48] "00073.4bd106f62a13d9071a638228a7189862"
 [49] "00074.c6f6a8cef7af2b318f769fc4633aa68d"
 [50] "00075.7e1dad2c4dde79ba419379fcad78f2ca"
 [51] "00076.5775c01eef4784032d58e85f76c541da"
 [52] "00077.5bb73a24591aea238ff4c466aa16c6a5"
 [53] "00080.338dc467b34650c3fb2deebfb49c7279"
 [54] "00081.a50816e80775062cf2b291e66137d01b"
 [55] "00082.a405e76faf9463d464229306b1e0c93f"
 [56] "00085.661f10298bde7a7c873f9d90fc343d4d"
 [57] "00086.5c80a9cdcc27b05861d4ccef262ddab7"
 [58] "00087.4563348fbfa23de502fee878d59ec671"
 [59] "00092.dea3df11dafb2cdd838b42fc3b9ead09"
 [60] "00093.d3befc62e479414b36e08add1a79bed1"
 [61] "00094.2775efe4f46c64a5ec4af86872a1548f"
 [62] "00095.8649aca72a33fe998c9b95ad00dc4d7f"
 [63] "00096.bfde9cb2286020360aea7e6e5dcbb39e"
 [64] "00097.90200a177414673b08df827239a0b9dc"
 [65] "00099.83f69964f7132d2c244a3374b94c285e"
 [66] "00100.78af3dc4c39277a6e1893f287cc2771f"
 [67] "00101.35e2f90cbe45708b1c1dcb606b554ca7"
 [68] "00102.10533c2a8486368eea6acd0d07549f3b"
 [69] "00103.3c2ed4232a349875b6ba4d78a9a7eb08"
 [70] "00106.8325a1afdd64c2945889fc4b0e80d363"
 [71] "00107.88cccf3d99a841ce92bdff029e137ed9"
 [72] "00109.869c6cf86d562eed3692f760e9320cc0"
 [73] "00110.77ac47048cafa2fe2587182f6d6b19d2"
 [74] "00111.9b8da30db6709b590398fe8923a7180c"
 [75] "00115.951fbc0e7de3d8ae828928279e43175c"
 [76] "00117.cf6312eae6441d25bef2ecfc39cc4acb"
 [77] "00118.afcf98c4f10a53e86d033ddf95660b5d"
 [78] "00119.4cb9c1c20358b4c2a6df2cdc5670fe62"
 [79] "00120.4614aadda75cb2e069a6c22cabc5aea0"
 [80] "00121.92d8827f2a5d552697f3bd261e6c0399"
 [81] "00122.c0299a711b7392c9e93da37cce2fdc90"
 [82] "00123.d9456f172d3424987c46cb49b7b6b012"
 [83] "00124.ac778f6679d31ae64aae6fdb88327fbb"
 [84] "00127.ab907b37736fd663c0db15b5575cc416"
 [85] "00128.d93c580cbc23b080e501f3ea11dbaa5e"
 [86] "

[1] "Nombres de easy ham mal clasificados"


[1] "01325.d94c9e1cca235f9f1bcecf469954490f"

[1] "Nombres de spam mal clasificados"


[1] "00007.acefeee792b5298f8fee175f9f65c453"
 [2] "00172.0935a6d0aef9a3d6d64e07e3f6c453ec"
 [3] "00174.94a8f3a8826ff937c38640422784ce86"
 [4] "00191.514dbda1bd10302e5c1d269837df0c66"
 [5] "00195.aeb47d08c2d537c24ada260804b7a171"
 [6] "00226.bd5d6adde04612587537df13d7f81196"
 [7] "00288.e1f7256dd7c447815e6693fd8efea6c9"
 [8] "00388.a884c42d4423d7e4718db0145b3b9d9b"
 [9] "00408.78e871a38b048989b9949716e3fc7575"
[10] "00438.cf76c0c71830d5e8ddec01a597f149a5"
[11] "00459.2c2341d0342bfedc94464025126bedc6"
[12] "00529.0c8a07bb7b14576063ba0c1c4079e209"
[13] "00724.edc5a3ce2ea91fedbf00aa0c30459a87"
[14] "00757.c2da75286819a139e27438ca5c5ba762"
[15] "00775.f83fab6722ac0be1f4231f8fb3845007"
[16] "00787.6ac0d6fe5aa9e89ee18e98ed8a556895"
[17] "00822.582f541f39bb9e92cb1262875d3a9fca"
[18] "01022.55c9eda45ef3de55b8c27c214a1fc305"
[19] "01152.3cd924b7f65e2085150c613cfe2b8c42"
[20] "01321.f32dcf9a564da7e27f9fcad664cb0fe1"
[21] "01361.1b094e93a9a83d77859dcbf6637455d1"

In [21]:
# Palabras con mayor probabilidad en los correos mal clasificados 

hard_ham_res_2 <- unlist(hard_ham_res[2], FALSE, use.names = FALSE)
print("Palabras principales de hard ham mal clasificados")
hard_ham_res_2[hard_ham_res_1]

easy_ham_res_2 <- unlist(easy_ham_res[2], FALSE, use.names = FALSE)
print("Palabras principales de easy ham mal clasificados")
easy_ham_res_2[easy_ham_res_1]

spam_res_2 <- unlist(spam_res[2], FALSE, use.names = FALSE)
print("Palabras principales de spam mal clasificados")
spam_res_2[!spam_res_1]


[1] "Palabras principales de hard ham mal clasificados"


[[1]]
[1] "won"     "writing" "youve"  

[[2]]
[1] "renew"  "will"   "within"

[[3]]
[1] "widthd"   "wireless" "wizard"  

[[4]]
[1] "year"  "yet"   "youre"

[[5]]
[1] "years" "youll" "youre"

[[6]]
[1] "worldcom" "years"    "york"    

[[7]]
[1] "win"    "winner" "youll" 

[[8]]
[1] "widthdimg" "workbr"    "youre"    

[[9]]
[1] "youll" "youre" "youve"

[[10]]
[1] "work"   "works"  "worthy"

[[11]]
[1] "write" "youre" "zone" 

[[12]]
[1] "write" "years" "yet"  

[[13]]
[1] "widthtdtr" "windows"   "wireless" 

[[14]]
[1] "widthtd" "work"    "youve"  

[[15]]
[1] "world"    "worldcom" "years"   

[[16]]
[1] "worldcom" "year"     "years"   

[[17]]
[1] "world" "xxx"   "youre"

[[18]]
[1] "widthtd"             "widthtdtr"           "widthtdtrtbodytable"

[[19]]
[1] "works" "write" "youre"

[[20]]
[1] "youd"  "youll" "youre"

[[21]]
[1] "work"  "works" "wrote"

[[22]]
[1] "widthtdtr"           "widthtdtrtbodytable" "without"            

[[23]]
[1] "works" "yea"   "youll"

[[24]]
[1] "widthdimg" "workbr"    "youre"    

[[25]]
[1] "wrong" "years" "youre"

[[26]]
[1] "tel"       "textplain" "wap"      

[[27]]
[1] "yet"   "youre" "youve"

[[28]]
[1] "youll" "youre" "youve"

[[29]]
[1] "widthimg" "wireless" "world"   

[[30]]
[1] "width"    "widthimg" "widthtd" 

[[31]]
[1] "worries" "worry"   "youre"  

[[32]]
[1] "worldcom" "written"  "yahoo"   

[[33]]
[1] "winners"   "worldwide" "yahoo"    

[[34]]
[1] "years" "yet"   "young"

[[35]]
[1] "will"     "wireless" "works"   

[[36]]
[1] "works" "xxx"   "youre"

[[37]]
[1] "yet"   "youll" "youre"

[[38]]
[1] "widthdimg" "workbr"    "youre"    

[[39]]
[1] "worth" "write" "youre"

[[40]]
[1] "world" "xml"   "yet"  

[[41]]
[1] "wish"  "youll" "youre"

[[42]]
[1] "yahoo" "years" "yet"  

[[43]]
[1] "widthdimg" "workbr"    "youre"    

[[44]]
[1] "widthimg" "worse"    "youre"   

[[45]]
[1] "within"  "without" "word"   

[[46]]
[1] "windows" "wishes"  "youd"   

[[47]]
[1] "wrote" "year"  "youre"

[[48]]
[1] "worldcom" "years"    "yet"     

[[49]]
[1] "youll" "youre" "youve"

[[50]]
[1] "year"  "youll" "youve"

[[51]]
[1] "will"    "winners" "years"  

[[52]]
[1] "yahoo" "years" "yet"  

[[53]]
[1] "work"    "works"   "writing"

[[54]]
[1] "winner" "xerox"  "youll" 

[[55]]
[1] "write" "year"  "years"

[[56]]
[1] "wireless" "years"    "york"    

[[57]]
[1] "width"    "widthimg" "widthtd" 

[[58]]
[1] "windows"  "winners"  "wireless"

[[59]]
[1] "widthdimg" "workbr"    "youre"    

[[60]]
[1] "year"  "youll" "youp" 

[[61]]
[1] "wireless" "wish"     "youre"   

[[62]]
[1] "worried" "year"    "youre"  

[[63]]
[1] "youa" "youd" "zone"

[[64]]
[1] "years"     "yesterday" "yet"      

[[65]]
[1] "worst" "yahoo" "years"

[[66]]
[1] "worlds" "year"   "york"  

[[67]]
[1] "year"  "yet"   "youre"

[[68]]
[1] "worlds" "yet"    "youre" 

[[69]]
[1] "xml"   "yield" "zip"  

[[70]]
[1] "widtha"    "widthfont" "widthimg" 

[[71]]
[1] "works" "yet"   "youll"

[[72]]
[1] "work"    "works"   "writing"

[[73]]
[1] "write" "yahoo" "year" 

[[74]]
[1] "widthdimg" "workbr"    "youre"    

[[75]]
[1] "worst" "wrong" "york" 

[[76]]
[1] "year" "york" "zero"

[[77]]
[1] "youll" "youre" "youve"

[[78]]
[1] "worlds" "year"   "years" 

[[79]]
[1] "widthtdtr"           "widthtdtrtbodytable" "without"            

[[80]]
[1] "working" "works"   "yahoo"  

[[81]]
[1] "widthbr"  "widthimg" "widthtd" 

[[82]]
[1] "word"    "yearold" "york"   

[[83]]
[1] "wireless" "wonder"   "years"   

[[84]]
[1] "widthtd"  "wireless" "youve"   

[[85]]
[1] "wish"    "wonder"  "writing"

[[86]]
[1] "youll" "youre" "youve"

[[87]]
[1] "write"  "writes" "yes"   

[[88]]
[1] "widthdimg" "workbr"    "youre"    

[[89]]
[1] "year"  "years" "york" 

[[90]]
[1] "widthdimg" "workbr"    "youre"    

[[91]]
[1] "world" "xml"   "years"

[[92]]
[1] "widthdimg" "workbr"    "youre"    

[[93]]
[1] "youd"  "youre" "youve"

[[94]]
[1] "worries" "wrong"   "year"   

[[95]]
[1] "working"  "worldcom" "york"    

[[96]]
[1] "web"     "without" "world

[1] "Palabras principales de easy ham mal clasificados"


[[1]]
[1] "textplain"      "transitionalen" "wcdtd"

[1] "Palabras principales de spam mal clasificados"


[[1]]
[1] "winxwinntwink" "wish"          "works"        

[[2]]
[1] "write"                        "writing"                     
[3] "wwwanteecomsamplesandfaqhtml"

[[3]]
[1] "will"    "without" "working"

[[4]]
[1] "warp"    "without" "younger"

[[5]]
[1] "week"                         "will"                        
[3] "wwwpiercebrosnanlookalikecom"

[[6]]
[1] "wet"      "wide"     "wondered"

[[7]]
[1] "users"    "within"   "wsending"

[[8]]
[1] "testing"                 "wbxvkhiessomodyjybffiyl"
[3] "works"                  

[[9]]
[1] "year"  "years" "yrs"  

[[10]]
[1] "updates" "want"    "watch"  

[[11]]
[1] "working" "worlds"  "yet"    

[[12]]
[1] "writing" "wrote"   "youve"  

[[13]]
[1] "west" "will" "year"

[[14]]
[1] "users"    "within"   "wsending"

[[15]]
[1] "word"  "work"  "years"

[[16]]
[1] "unique"  "wanted"  "website"

[[17]]
[1] "visit" "will"  "world"

[[18]]
[1] "updates"          "want"             "wwwlinuxoemsinfo"

[[19]]
[1] "unsubscription" "users"          "verizon"       

[[20]]
[1] "unsubscription" "users"          "weekly"        

[[21]]
[1] "videos" "web"    "work"

In [22]:
summary(hard_ham_res_1)
summary(easy_ham_res_1)
summary(spam_res_1)


   Mode   FALSE    TRUE 
logical     116     134 

   Mode   FALSE    TRUE 
logical    1399       1 

   Mode   FALSE    TRUE 
logical      21    1376 

# Análisis

Dados los resultados obtenidos, el modelo clasifica bien los correos en la medida de lo posible, ya 
que el porcentaje de precisión se encuentra por arriba del 90%.
Sin embargo, cuando los correos se vuelven más complejos e incluyen elementos como html, como es el caso del conjunto hard ham, el
modelo comienza a disminuir la eficacia de su clasificación.